# <span style="color:seagreen">00-Restructure_data</span>

In this notebook we will guide you through restructuring your data. For the smFISH a particular folder and filename structure is expected. We will use this notebook to structure your data according to this format.

***

## 0.0 - Loading data

In [2]:
from glob import glob
from pathlib import Path
from skimage import io
import warnings
warnings.filterwarnings('ignore')

## 0.1 - Loading path to images

First we will load the paths to our images in the variable "image_paths". You will also have to provide the order of channels epxected in each fluorescent image stack.

In [3]:
#channel indexes, provide the right channel order. The default order is standardly given
CY5_id, CY35_id, DAPI_id = 0,1,2 

indexes=[CY5_id,CY35_id,DAPI_id]

In [5]:
path=f'../Data/replicate1/HWP1_CLB2/*/*/*.tif'
image_paths=sorted(glob(path))
image_paths

[]

***

## 0.2 - Restructure data

All data will be renamed and put in a single folder in "../Data/restructured_data/replicate1". 

In [ ]:
savedir=f'../Data/restructured_data'

i=1
for image_path in image_paths:
    #progress statement
    print(f'processing image {i} of {len(image_paths)}')
    i+=1
    
    # folder structure properties
    p=Path(image_path)
    
    #get replicate mRNAs and condition from provided folder structure
    rep,mRNA,condition,fov,image_name=p.parts[2:]
    
    # make replicate dir to save restructured data in
    rep_dir=Path(savedir,rep)
    rep_dir.mkdir(parents=True, exist_ok=True)
    
    #in case of single mRNA FISH
    if len(mRNA.split('_'))==1:
        # split image name properties
        strain,mRNA_CY5,condition,fov=image_name.split('_')[:5]
        mRNAs=mRNA_CY5
        
    #in case of dual mRNA FISH
    if len(mRNA.split('_'))==2:
        # split image name properties
        strain,mRNA_CY5,mRNA_CY35,condition,fov=image_name.split('_')[:5]
        mRNAs=mRNA_CY5+mRNA_CY35

    # saving for DIC
    if image_name.split('_')[-1]=='DIC.tif':
        
        #read DIC image into memory
        print('reading DIC image')
        DIC=io.imread(image_path)
        
        #reformatted name as expected for smFISH analysis
        formatted_name=f'{strain}_{mRNAs}_{condition}_DIC_{fov}.tif'
        
        #print reformatted name
        print(formatted_name)
        
        #save reformatted DIC images
        io.imsave(Path(rep_dir,formatted_name),DIC)
        
    # saving for fluorescent images
    if image_name.split('_')[-1]=='CY5, CY3.5 NAR, DAPI.tif':
                
        #read fluorescent image into memory
        print('reading fluorescent image stack')
        FC=io.imread(image_path)
        
        channels=['CY5','CY35','DAPI']
        
        print('splitting channels:')
        
        #split channels of fluorescent image and save seperately
        for index, channels in zip(indexes,channels):
            
            #select channel of multi-channel image stack
            FC_single_channel=FC[index,...]
            
            #reformat name
            formatted_name=f'{strain}_{mRNAs}_{condition}_{channels}_{fov}.tif'
            
            #print reformatted name
            print(formatted_name)
            
            #save 
            io.imsave(Path(rep_dir,formatted_name),FC_single_channel)
    
    print(' '*70)
    print('#'*70)
    print(' '*70)

# 0.3- Create necessary folder for analysis 

For the spot detection analysis we need to create cell masks and nuclear masks. During the creation of these masks and when performing smFISH spot quantification, several intermediate steps are saved. These folders are already created here.

In [ ]:
for image_path in image_paths:    
    # folder structure properties
    p=Path(image_path)
    rep,mRNA,condition,fov,image_name=p.parts[2:]
    
    # make replicate dir
    rep_dir=Path(savedir,rep)
    rep_dir.mkdir(parents=True, exist_ok=True)
    
    # make necessary dirs
    dirs=['Spots','Masks','Spots decomposition','Zprojection','Results','Corrected DIC']
    for ndir in dirs:
        necessary_dir=Path(rep_dir,ndir)
        necessary_dir.mkdir(parents=True, exist_ok=True)